<a href="https://colab.research.google.com/github/vishnusnath447/BYOB/blob/main/byob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers flask torch

In [5]:
!pip uninstall compressed-tensors -y
!pip install compressed-tensors

Found existing installation: compressed-tensors 0.8.1
Uninstalling compressed-tensors-0.8.1:
  Successfully uninstalled compressed-tensors-0.8.1
  Using cached compressed_tensors-0.8.1-py3-none-any.whl.metadata (6.8 kB)
Using cached compressed_tensors-0.8.1-py3-none-any.whl (87 kB)


In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the quantized model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load data from JSON
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Save default JSON files if none exist
def create_default_files():
    default_tasks = {
        "tasks": [
            {"description": "Complete the project report", "status": "pending"},
            {"description": "Prepare slides for the meeting", "status": "pending"},
            {"description": "Review the budget for Q4", "status": "completed"}
        ]
    }
    with open(tasks_file, "w") as file:
        json.dump(default_tasks, file, indent=4)

    default_meetings = {
        "meetings": [
            {
                "title": "Team Sync-Up",
                "date": "2025-01-05",
                "time": "10:00 AM",
                "is_recording": True,
                "agenda": "Discuss project progress and blockers."
            },
            {
                "title": "Client Presentation",
                "date": "2025-01-06",
                "time": "02:00 PM",
                "is_recording": False,
                "agenda": "Present Q4 results and proposed strategy."
            }
        ]
    }
    with open(meetings_file, "w") as file:
        json.dump(default_meetings, file, indent=4)

# Ensure JSON files exist
try:
    tasks = load_json(tasks_file)
except FileNotFoundError:
    create_default_files()
    tasks = load_json(tasks_file)

try:
    meetings = load_json(meetings_file)
except FileNotFoundError:
    create_default_files()
    meetings = load_json(meetings_file)

# Function to filter and format pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Function to format upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Function to generate responses with task and meeting context
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)  # Move inputs to GPU
    outputs = model.generate(
        **inputs,
        max_length=300,           # Increase max_length to allow a longer response
        num_return_sequences=1,   # Generate a single response
        temperature=0.7,          # Adjust creativity of the response
        top_p=0.9,                # Use nucleus sampling
        top_k=50,                 # Limit to top-k sampling
        repetition_penalty=1.2    # Penalize repetitive tokens
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Trim response to exclude the prompt
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Example usage
user_input = "What meetings do I have this week and what tasks are pending?"
response = generate_response(user_input, tasks, meetings)
print(response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


This week, you have two meetings scheduled:
1. Team Sync-Up on 2025-01-05 at 10:00 AM. This meeting will be recorded. The agenda is to discuss project progress and blockers.
2. Client Presentation on 2025-01-06 at 02:00 PM. This meeting will not be recorded. The agenda is to present Q4 results and propose strategies.

In addition to these meetings, there are some pending tasks that need your attention:
- Complete the project report
- Prepare slides for the meeting


In [ ]:
user_input = "suggest what all topics can be covered on 05/01/2025 's meeting."
response = generate_response(user_input, tasks, meetings)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Topics that can be covered during the Team Sync-Up on 05/01/2025 include:
1. Project Progress Update - Share updates on the current status of each task assigned within the team.
2. Blocker Identification & Resolution - Address any obstacles or challenges faced by team members in completing their tasks.
3. Action Items Assignment - Distribute new action items based on the discussion points from this meeting.
4. Upcoming Deadlines - Review important deadlines coming up in the near future and ensure everyone is aware of them.
5. Communication Plan - Establish


# **BACKEND CODE**

In [2]:
# Install Flask if not already installed
!pip install flask flask-cors

import torch
import json
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the quantized model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load JSON data
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Ensure JSON files exist
tasks = load_json(tasks_file)
meetings = load_json(meetings_file)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [1]:
import threading
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

# Load the model and tokenizer
model_name = "neuralmagic/Mistral-7B-Instruct-v0.3-quantized.w8a8"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# File paths
tasks_file = "tasks.json"
meetings_file = "meetings.json"

# Load JSON data
def load_json(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Load tasks and meetings data
tasks = load_json(tasks_file)
meetings = load_json(meetings_file)

# Get pending tasks
def get_pending_tasks(tasks):
    pending_tasks = [task["description"] for task in tasks["tasks"] if task["status"] == "pending"]
    if not pending_tasks:
        return "You have no pending tasks!"
    return "\n".join(f"- {task}" for task in pending_tasks)

# Get upcoming meetings
def get_upcoming_meetings(meetings):
    upcoming_meetings = [
        f"{meeting['title']} on {meeting['date']} at {meeting['time']}. "
        f"Agenda: {meeting['agenda']}. Recording: {'Yes' if meeting['is_recording'] else 'No'}"
        for meeting in meetings["meetings"]
    ]
    if not upcoming_meetings:
        return "You have no upcoming meetings!"
    return "\n".join(f"- {meeting}" for meeting in upcoming_meetings)

# Generate response
def generate_response(prompt, tasks, meetings):
    pending_tasks_summary = get_pending_tasks(tasks)
    upcoming_meetings_summary = get_upcoming_meetings(meetings)
    manager_prompt = (
        f"You are a virtual manager. The user relies on you for tracking tasks and meetings. "
        f"Here are the pending tasks:\n{pending_tasks_summary}\n\n"
        f"Here are the upcoming meetings:\n{upcoming_meetings_summary}\n\n"
        f"Now, respond to the user's query: {prompt}"
    )
    inputs = tokenizer(manager_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=300,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_output = response[len(manager_prompt):].strip()
    return generated_output

# Flask API setup
app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    prompt = data.get('prompt', '')
    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400
    response = generate_response(prompt, tasks, meetings)
    return jsonify({'response': response})

# Run Flask in a background thread
def run_server():
    app.run(host='0.0.0.0', port=5000)

server_thread = threading.Thread(target=run_server)
server_thread.daemon = True
server_thread.start()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [3]:
!ngrok authtoken 2r4wIMeTofYcdj9x1dmfbNPHckc_6sKrTQ5ovSVKJDR43e82X

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
!pip install pyngrok
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(5000)
print("Public URL:", ngrok_tunnel.public_url)

Public URL: https://fbff-34-75-202-93.ngrok-free.app
